In [1]:
# ============================
# library
# ============================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import LayerNorm,TransformerEncoder
from torch.utils.data import DataLoader, TensorDataset, Dataset
import pickle
from tqdm import tqdm
import gc
import glob
from sklearn.preprocessing import StandardScaler,RobustScaler

In [2]:
# ============================
# constant
# ============================
SUB_PATH = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/sample_submission.csv"
DEFOG_DATA_PATH = "/kaggle/working/acceleration_data.csv"
TDCSFOG_DATA_PATH = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/*.csv"

In [3]:
# ============================
# settings
# ============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bs = 32

In [4]:
# ============================
# settings
# ============================
tdcsfog_path1 = [f"/kaggle/input/fog-ex143/ex143_{i}.pth" for i in range(5)] # len 3000 cv TdcsfogRnnModel cv
tdcsfog_path3_1 = [f"/kaggle/input/fog-ex145/ex145_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel  
tdcsfog_path3_2 = [f"/kaggle/input/fog-ex146/ex146_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel 
tdcsfog_path3_3 = [f"/kaggle/input/fog-ex147/ex147_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel cv
tdcsfog_path4_1 = [f"/kaggle/input/fog-ex182/ex182_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel  
tdcsfog_path4_2 = [f"/kaggle/input/fog-ex183/ex183_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel 
tdcsfog_path4_3 = [f"/kaggle/input/fog-ex184/ex184_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel cv

defog_path2 = [f"/kaggle/input/fog-ex153/ex153_{i}.pth" for i in range(5)] # len 30000 defog1 
defog_path4 = [f"/kaggle/input/fog-ex179/ex179_{i}.pth" for i in range(5)] # len 30000 defog1
defog_path5 = [f"/kaggle/input/fog-ex185/ex185_{i}.pth" for i in range(5)] # len 30000 defog1
defog_path6 = [f"/kaggle/input/fog-ex204/ex204_{i}.pth" for i in range(5)] # len 30000 defog2
defog_path7 = [f"/kaggle/input/pd-exp238/fold{i}_best.pth" for i in [0, 1, 2, 3, 4]]  # len 30000 Defog3Model

In [5]:
# ============================
# Functions
# ============================
def preprocess(numerical_array, 
               mask_array,
               ):
    
    attention_mask = mask_array == 0

    return {
        'input_data_numerical_array': numerical_array,
        'input_data_mask_array': mask_array,
        'attention_mask': attention_mask,
    }

class FogDataset(Dataset):
    def __init__(self, numerical_array, 
                 mask_array,
                 train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.train = train
        self.y = y
    
    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(
            self.numerical_array[item],
            self.mask_array[item],
            
        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y":torch.tensor(self.y[item], dtype=torch.float32)
               }
        else:
            return {
             'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

In [6]:
# ================================
# tdcsfog
# ================================
class TdcsfogRnnModel(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=12,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(TdcsfogRnnModel, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
    
class TdcsfogRnnModel2(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=12,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(TdcsfogRnnModel2, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout))
        self.out1 = nn.Linear(linear_out, 
                          out_size)
        self.out2 = nn.Linear(linear_out, 
                          out_size)

        
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        output1 = self.out1(output)
        output2 = self.out2(output)
        return output1,output2

In [7]:
class DefogRnnModel(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(DefogRnnModel, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
    
class DefogRnnModel2(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size = 96,
        model_size = 256,
        linear_out = 256,
        out_size=3):
        super(DefogRnnModel2, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
    
class Defog3Model(nn.Module):
    def __init__(
        self,
        dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size=64,
        model_size=128,
        linear_out=128,
        out_size=3,
    ):
        super(Defog3Model, self).__init__()
        self.numerical_linear = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        self.lstm = nn.GRU(
            numeraical_linear_size,
            model_size,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
        )
        self.linear_out = nn.Sequential(
            nn.Linear(model_size*2, linear_out),
            nn.LayerNorm(linear_out),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(linear_out, out_size),
        )
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)

    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.lstm(numerical_embedding)
        output = self.linear_out(output)
        return output

In [8]:
def make_pred(test_loader,model):
    test_preds = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    with torch.no_grad():  # Do not calculate gradient since we are only predicting
        # Predicting on validation set
        for d in tk0:
            input_data_numerical_array = d['input_data_numerical_array'].to(device)
            input_data_mask_array = d['input_data_mask_array'].to(device)
            attention_mask = d['attention_mask'].to(device)
            output = model(input_data_numerical_array, 
                       input_data_mask_array,
                       attention_mask)
            test_preds.append(output.sigmoid().cpu().numpy())
    test_preds = np.concatenate(test_preds,axis=0)
    return test_preds


def make_pred2(test_loader,model):
    test_preds = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    with torch.no_grad():  # Do not calculate gradient since we are only predicting
        # Predicting on validation set
        for d in tk0:
            input_data_numerical_array = d['input_data_numerical_array'].to(device)
            input_data_mask_array = d['input_data_mask_array'].to(device)
            attention_mask = d['attention_mask'].to(device)
            output,_ = model(input_data_numerical_array, 
                       input_data_mask_array,
                       attention_mask)
            test_preds.append(output.sigmoid().cpu().numpy())
    test_preds = np.concatenate(test_preds,axis=0)
    return test_preds

In [9]:
# =======================
# main
# =======================
sub = pd.read_csv(SUB_PATH)

In [10]:
# Helper function for loading models with map_location set
def load_model(model_class, path, device):
    model = model_class()
    try:
        state_dict = torch.load(path, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict)
        model = model.to(device)
        model.eval()
        print(f"Successfully loaded model from {path}")
    except Exception as e:
        print(f"Error loading model from {path}: {e}")
    return model

# ===========================
# tdcsfog models
# ===========================
tdcsfog_model_list1 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path1]
tdcsfog_model_list3_1 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path3_1]
tdcsfog_model_list3_2 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path3_2]
tdcsfog_model_list3_3 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path3_3]
tdcsfog_model_list4_1 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path4_1]
tdcsfog_model_list4_2 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path4_2]
tdcsfog_model_list4_3 = [load_model(TdcsfogRnnModel, i, device) for i in tdcsfog_path4_3]

# ===========================
# defog models
# ===========================
defog_model_list2 = [load_model(DefogRnnModel, i, device) for i in defog_path2]
defog_model_list4 = [load_model(DefogRnnModel, i, device) for i in defog_path4]
defog_model_list5 = [load_model(DefogRnnModel, i, device) for i in defog_path5]
defog_model_list6 = [load_model(DefogRnnModel2, i, device) for i in defog_path6]

# For defog_model_list7, using state dictionary:
def load_defog3_model(path, device):
    model = Defog3Model()
    try:
        state = torch.load(path, map_location=torch.device("cpu"))
        model.load_state_dict(state["model"])
        model = model.to(device)
        model.eval()
        print(f"Successfully loaded weights from {path}")
    except Exception as e:
        print(f"Error loading weights from {path}: {e}")
    return model

defog_model_list7 = [load_defog3_model(i, device) for i in defog_path7]

/tmp/ipykernel_30/310761028.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=torch.device("cpu"))


Successfully loaded model from /kaggle/input/fog-ex143/ex143_0.pth
Successfully loaded model from /kaggle/input/fog-ex143/ex143_1.pth
Successfully loaded model from /kaggle/input/fog-ex143/ex143_2.pth
Successfully loaded model from /kaggle/input/fog-ex143/ex143_3.pth
Successfully loaded model from /kaggle/input/fog-ex143/ex143_4.pth
Successfully loaded model from /kaggle/input/fog-ex145/ex145_0.pth
Successfully loaded model from /kaggle/input/fog-ex145/ex145_1.pth
Successfully loaded model from /kaggle/input/fog-ex145/ex145_2.pth
Successfully loaded model from /kaggle/input/fog-ex145/ex145_3.pth
Successfully loaded model from /kaggle/input/fog-ex145/ex145_4.pth
Successfully loaded model from /kaggle/input/fog-ex146/ex146_0.pth
Successfully loaded model from /kaggle/input/fog-ex146/ex146_1.pth
Successfully loaded model from /kaggle/input/fog-ex146/ex146_2.pth
Successfully loaded model from /kaggle/input/fog-ex146/ex146_3.pth
Successfully loaded model from /kaggle/input/fog-ex146/ex146_4

/tmp/ipykernel_30/310761028.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, map_location=torch.device("cpu"))


Successfully loaded weights from /kaggle/input/pd-exp238/fold4_best.pth


In [11]:
df_all = []

# defog

In [12]:
# =========================
def defog2():
# =========================
    th_len = 200000
    w = 0.35
    
    defog_list = glob.glob(DEFOG_DATA_PATH)
    cols = ["AccV","AccML","AccAP"]
    num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
                'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
                'AccAP_lag_diff', 'AccAP_lead_diff']
    for p in tqdm(defog_list):
        id_values = p.split("/")[-1].split(".")[0]
        df = pd.read_csv(p)
        if len(df) > th_len:
            seq_len = 30000
            shift = 15000
            offset = 7500
        else:
            seq_len = 15000
            shift = 7500
            offset = 3750
        batch = (len(df)-1) // shift
        if batch == 0:
            batch = 1
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)
        num = df[num_cols].values
        time = df["Time"].values
        
        num_array = np.zeros([batch,seq_len,9])
        mask_array = np.zeros([batch,seq_len],dtype=int)
        time_array = np.zeros([batch,seq_len],dtype=int)
        pred_use_array = np.zeros([batch,seq_len],dtype=int)
        
        if len(df) <= seq_len:
            b = 0
            num_len = len(num)
            num_array[b,:num_len,:] = num
            time_array[b,:num_len] = time
            mask_array[b,:num_len] = 1
            pred_use_array[b,:num_len] = 1
        else:
            for n,b in enumerate(range(batch)):
                if b == (batch - 1):
                    num_ = num[b*shift : ]
                    time_ = time[b*shift : ]
                    num_len = len(num_)
    
                    num_array[b,:num_len,:] = num_
                    time_array[b,:num_len] = time_
                    mask_array[b,:num_len] = 1
                    pred_use_array[b,offset:num_len] = 1
                elif b == 0:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,:shift+offset] = 1
                else:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,offset:shift+offset] = 1  
        
        test_ = FogDataset(num_array,
                           mask_array,
                           train=False)
        test_loader = DataLoader(dataset=test_, 
                            batch_size=bs, 
                            shuffle = False)
        for n,m in enumerate(defog_model_list2):
            if n == 0:
                pred = make_pred(test_loader,m) / len(defog_model_list2)
            else:
                pred += make_pred(test_loader,m) / len(defog_model_list2)
        pred_list = []
        for i in range(batch):
            mask_ = pred_use_array[i]
            pred_ = pred[i,mask_ == 1,:]
            time_ = time_array[i, mask_ == 1]
            df_ = pd.DataFrame()
            df_["StartHesitation"] = pred_[:,0] * w
            df_["Turn"] = pred_[:,1] * w
            df_["Walking"] = pred_[:,2] * w
            df_["Time"] = time_
            df_["Id"] = id_values
            df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
            pred_list.append(df_)
        pred = pd.concat(pred_list).reset_index(drop=True)
        df_all.append(pred)

In [13]:
# =========================
def defog4():
# =========================
    th_len = 200000
    w = 0.25
    
    defog_list = glob.glob(DEFOG_DATA_PATH)
    cols = ["AccV","AccML","AccAP"]
    num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
                'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
                'AccAP_lag_diff', 'AccAP_lead_diff']
    for p in tqdm(defog_list):
        id_values = p.split("/")[-1].split(".")[0]
        df = pd.read_csv(p)
        if len(df) > th_len:
            seq_len = 30000
            shift = 15000
            offset = 7500
        else:
            seq_len = 15000
            shift = 7500
            offset = 3750
        batch = (len(df)-1) // shift
        if batch == 0:
            batch = 1
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)
        num = df[num_cols].values
        time = df["Time"].values
        
        num_array = np.zeros([batch,seq_len,9])
        mask_array = np.zeros([batch,seq_len],dtype=int)
        time_array = np.zeros([batch,seq_len],dtype=int)
        pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
        if len(df) <= seq_len:
            b = 0
            num_len = len(num)
            num_array[b,:num_len,:] = num
            time_array[b,:num_len] = time
            mask_array[b,:num_len] = 1
            pred_use_array[b,:num_len] = 1
        else:
            for n,b in enumerate(range(batch)):
                if b == (batch - 1):
                    num_ = num[b*shift : ]
                    time_ = time[b*shift : ]
                    num_len = len(num_)
    
                    num_array[b,:num_len,:] = num_
                    time_array[b,:num_len] = time_
                    mask_array[b,:num_len] = 1
                    pred_use_array[b,offset:num_len] = 1
                elif b == 0:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,:shift+offset] = 1
                else:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,offset:shift+offset] = 1  
        
        test_ = FogDataset(num_array,
                           mask_array,
                           train=False)
        test_loader = DataLoader(dataset=test_, 
                            batch_size=bs, 
                            shuffle = False)
        for n,m in enumerate(defog_model_list4):
            if n == 0:
                pred = make_pred(test_loader,m) / len(defog_model_list4)
            else:
                pred += make_pred(test_loader,m) / len(defog_model_list4)
        pred_list = []
        for i in range(batch):
            mask_ = pred_use_array[i]
            pred_ = pred[i,mask_ == 1,:]
            time_ = time_array[i, mask_ == 1]
            df_ = pd.DataFrame()
            df_["StartHesitation"] = pred_[:,0] * w
            df_["Turn"] = pred_[:,1] * w
            df_["Walking"] = pred_[:,2] * w
            df_["Time"] = time_
            df_["Id"] = id_values
            df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
            pred_list.append(df_)
        pred = pd.concat(pred_list).reset_index(drop=True)
        df_all.append(pred)

In [14]:
# =========================
def defog5():
# =========================
    th_len = 200000
    w = 0.25

    defog_list = glob.glob(DEFOG_DATA_PATH)
    cols = ["AccV","AccML","AccAP"]
    num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
                'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
                'AccAP_lag_diff', 'AccAP_lead_diff']
    for p in tqdm(defog_list):
        id_values = p.split("/")[-1].split(".")[0]
        df = pd.read_csv(p)
        if len(df) > th_len:
            seq_len = 30000
            shift = 15000
            offset = 7500
        else:
            seq_len = 15000
            shift = 7500
            offset = 3750
        batch = (len(df)-1) // shift
        if batch == 0:
            batch = 1
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)
        num = df[num_cols].values
        time = df["Time"].values
    
        num_array = np.zeros([batch,seq_len,9])
        mask_array = np.zeros([batch,seq_len],dtype=int)
        time_array = np.zeros([batch,seq_len],dtype=int)
        pred_use_array = np.zeros([batch,seq_len],dtype=int)
        
        if len(df) <= seq_len:
            b = 0
            num_len = len(num)
            num_array[b,:num_len,:] = num
            time_array[b,:num_len] = time
            mask_array[b,:num_len] = 1
            pred_use_array[b,:num_len] = 1
        else:
            for n,b in enumerate(range(batch)):
                if b == (batch - 1):
                    num_ = num[b*shift : ]
                    time_ = time[b*shift : ]
                    num_len = len(num_)
    
                    num_array[b,:num_len,:] = num_
                    time_array[b,:num_len] = time_
                    mask_array[b,:num_len] = 1
                    pred_use_array[b,offset:num_len] = 1
                elif b == 0:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,:shift+offset] = 1
                else:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,offset:shift+offset] = 1  
        
        test_ = FogDataset(num_array,
                           mask_array,
                           train=False)
        test_loader = DataLoader(dataset=test_, 
                            batch_size=bs, 
                            shuffle = False)
        for n,m in enumerate(defog_model_list5):
            if n == 0:
                pred = make_pred(test_loader,m) / len(defog_model_list5)
            else:
                pred += make_pred(test_loader,m) / len(defog_model_list5)
        pred_list = []
        for i in range(batch):
            mask_ = pred_use_array[i]
            pred_ = pred[i,mask_ == 1,:]
            time_ = time_array[i, mask_ == 1]
            df_ = pd.DataFrame()
            df_["StartHesitation"] = pred_[:,0] * w
            df_["Turn"] = pred_[:,1] * w
            df_["Walking"] = pred_[:,2] * w
            df_["Time"] = time_
            df_["Id"] = id_values
            df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
            pred_list.append(df_)
        pred = pd.concat(pred_list).reset_index(drop=True)
        df_all.append(pred)

In [15]:
# =========================
def defog6():
# =========================
    th_len = 200000
    w = 0.10
    
    defog_list = glob.glob(DEFOG_DATA_PATH)
    cols = ["AccV","AccML","AccAP"]
    num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
                'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
                'AccAP_lag_diff', 'AccAP_lead_diff']
    for p in tqdm(defog_list):
        id_values = p.split("/")[-1].split(".")[0]
        df = pd.read_csv(p)
        if len(df) > th_len:
            seq_len = 30000
            shift = 15000
            offset = 7500
        else:
            seq_len = 15000
            shift = 7500
            offset = 3750
        batch = (len(df)-1) // shift
        if batch == 0:
            batch = 1
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)
        num = df[num_cols].values
        time = df["Time"].values
        
        num_array = np.zeros([batch,seq_len,9])
        mask_array = np.zeros([batch,seq_len],dtype=int)
        time_array = np.zeros([batch,seq_len],dtype=int)
        pred_use_array = np.zeros([batch,seq_len],dtype=int)
        
        if len(df) <= seq_len:
            b = 0
            num_len = len(num)
            num_array[b,:num_len,:] = num
            time_array[b,:num_len] = time
            mask_array[b,:num_len] = 1
            pred_use_array[b,:num_len] = 1
        else:
            for n,b in enumerate(range(batch)):
                if b == (batch - 1):
                    num_ = num[b*shift : ]
                    time_ = time[b*shift : ]
                    num_len = len(num_)
    
                    num_array[b,:num_len,:] = num_
                    time_array[b,:num_len] = time_
                    mask_array[b,:num_len] = 1
                    pred_use_array[b,offset:num_len] = 1
                elif b == 0:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,:shift+offset] = 1
                else:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]
    
                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,offset:shift+offset] = 1  
        
        test_ = FogDataset(num_array,
                           mask_array,
                           train=False)
        test_loader = DataLoader(dataset=test_, 
                            batch_size=bs, 
                            shuffle = False)
        for n,m in enumerate(defog_model_list6):
            if n == 0:
                pred = make_pred(test_loader,m) / len(defog_model_list6)
            else:
                pred += make_pred(test_loader,m) / len(defog_model_list6)
        pred_list = []
        for i in range(batch):
            mask_ = pred_use_array[i]
            pred_ = pred[i,mask_ == 1,:]
            time_ = time_array[i, mask_ == 1]
            df_ = pd.DataFrame()
            df_["StartHesitation"] = pred_[:,0] * w
            df_["Turn"] = pred_[:,1] * w
            df_["Walking"] = pred_[:,2] * w
            df_["Time"] = time_
            df_["Id"] = id_values
            df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
            pred_list.append(df_)
        pred = pd.concat(pred_list).reset_index(drop=True)
        df_all.append(pred)

In [16]:
def run_predictions():
    # Now df_all is available for defog to use
    defog2()
    defog4()
    defog5()
    defog6()

    # =========================
    # defog7
    # =========================
    th_len = 200000
    w = 0.05
    
    submission_file = '/kaggle/working/submission.csv'
    
    defog_list = glob.glob(DEFOG_DATA_PATH)
    cols = ["AccV","AccML","AccAP"]
    num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
                'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
                'AccAP_lag_diff', 'AccAP_lead_diff']
    for p in tqdm(defog_list):
        id_values = p.split("/")[-1].split(".")[0]
        df = pd.read_csv(p)
        if len(df) > th_len:
            seq_len = 30000
            shift = 15000
            offset = 7500
        else:
            seq_len = 15000
            shift = 7500
            offset = 3750
        batch = (len(df)-1) // shift
        if batch == 0:
            batch = 1
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)
        num = df[num_cols].values
        time = df["Time"].values

        num_array = np.zeros([batch,seq_len,9])
        mask_array = np.zeros([batch,seq_len],dtype=int)
        time_array = np.zeros([batch,seq_len],dtype=int)
        pred_use_array = np.zeros([batch,seq_len],dtype=int)

        if len(df) <= seq_len:
            b = 0
            num_len = len(num)
            num_array[b,:num_len,:] = num
            time_array[b,:num_len] = time
            mask_array[b,:num_len] = 1
            pred_use_array[b,:num_len] = 1
        else:
            for n,b in enumerate(range(batch)):
                if b == (batch - 1):
                    num_ = num[b*shift : ]
                    time_ = time[b*shift : ]
                    num_len = len(num_)

                    num_array[b,:num_len,:] = num_
                    time_array[b,:num_len] = time_
                    mask_array[b,:num_len] = 1
                    pred_use_array[b,offset:num_len] = 1
                elif b == 0:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]

                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,:shift+offset] = 1
                else:
                    num_ = num[b*shift:b*shift+seq_len]
                    time_ = time[b*shift:b*shift + seq_len]

                    num_array[b,:,:] = num_
                    time_array[b,:] = time_
                    mask_array[b,:] = 1
                    pred_use_array[b,offset:shift+offset] = 1  

        test_ = FogDataset(num_array,
                           mask_array,
                           train=False)
        test_loader = DataLoader(dataset=test_, 
                            batch_size=bs, 
                            shuffle = False)
        for n,m in enumerate(defog_model_list7):
            if n == 0:
                pred = make_pred(test_loader,m)/len(defog_model_list7)
            else:
                pred += make_pred(test_loader,m)/len(defog_model_list7)
        pred_list = []
        for i in range(batch):
            mask_ = pred_use_array[i]
            pred_ = pred[i,mask_ == 1,:]  #len(defog_model_list7)
            time_ = time_array[i, mask_ == 1]
            df_ = pd.DataFrame()
            df_["StartHesitation"] = pred_[:,0] * w
            df_["Turn"] = pred_[:,1] * w
            df_["Walking"] = pred_[:,2] * w
            df_["Time"] = time_
            df_["Id"] = id_values
            df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
            pred_list.append(df_)
        pred = pd.concat(pred_list).reset_index(drop=True)
        df_all.append(pred)
        
    pd.concat(df_all).reset_index(drop=True).to_csv(submission_file, index=False)
    
    print(f"Predictions saved to {submission_file}")

    # Unpause data collection after predictions are done
    global paused
    paused = False
    print("Prediction complete. Resuming data collection...")

In [17]:
!pip install pyngrok

from pyngrok import ngrok      
#input NGROK to workspace, put encryption on the tunnel. You may need to regenerate an authentication code from fresh

# Replace 'your_ngrok_authtoken' with the actual authtoken from your ngrok dashboard
ngrok.set_auth_token("2nJxmviG9iZizA2kwtwzRnt5cTt_7aJbdy8tsmhQPq5tsxXkx")

# Start ngrok tunnel on port 8080
public_url = ngrok.connect(80)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://ec49-35-230-41-36.ngrok-free.app" -> "http://localhost:80"        


In [ ]:
import requests  # Add requests for HTTP POST
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import time
import csv
import os

# Parameters
required_data_length = 10  # 15000  # Number of data points required before triggering prediction
file_path = '/kaggle/working/acceleration_data.csv'
submission_file = '/kaggle/working/submission.csv'
save_interval = 3  # Time interval to save data
dataStore = {}
collected_data = []  # Store the data before predictions
last_saved_time = 0
start_time = None
paused = False

# Function to save data to CSV file
def save_data_to_csv(data):
    write_header = not os.path.exists(file_path)
    with open(file_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        if write_header:
            csv_writer.writerow(['Time', 'AccV', 'AccML', 'AccAP'])
        for row in data:
            csv_writer.writerow(row)

# HTTP Server
class MyHTTPRequestHandler(BaseHTTPRequestHandler):
    global collected_data, last_saved_time, paused

    def log_url(self):
        pathList = self.path.split("/")
        pathList.pop(0)
        print(f"URL visited: {pathList}")

    # Handle GET requests
    def do_GET(self):
        self.log_url()  # Log the URL

        key = self.path.lstrip('/')
        
        # To confirm that data has been sent
        if key in dataStore:
            response = {key: dataStore[key]}
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(response).encode('utf-8'))

        # To view current time stamp and acceleration data
        elif key == "":
            response = dataStore
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(response).encode('utf-8'))

        elif key == "download":
            try:
                with open(submission_file, 'r') as file:
                    response = file.read()
                self.send_response(200)
                self.send_header('Content-type', 'text/csv')
                self.end_headers()
                self.wfile.write(response.encode('utf-8'))
                
            except FileNotFoundError:
                response = {'error': 'File not found'}
                self.send_response(404)
                self.send_header('Content-type', 'application/json')
                self.end_headers()
                self.wfile.write(json.dumps(response).encode('utf-8'))
        
        # To view all acceleration data stored
        elif key == "acceld":
            try:
                with open(file_path, 'r') as file:
                    response = file.read()
                self.send_response(200)
                self.send_header('Content-type', 'text/csv')
                self.end_headers()
                self.wfile.write(response.encode('utf-8'))
                
            except FileNotFoundError:
                response = {'error': 'File not found'}
                self.send_response(404)
                self.send_header('Content-type', 'application/json')
                self.end_headers()
                self.wfile.write(json.dumps(response).encode('utf-8'))
                
        else:
            response = {'error': key}
            self.send_response(404)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(response).encode('utf-8'))


    # Handle POST requests for data collection
    def do_POST(self):
        global last_saved_time, start_time, paused, collected_data

        if paused:
            # Server is paused, waiting for predictions to finish
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            response = {"message": "Paused. Waiting for prediction result."}
            self.wfile.write(json.dumps(response).encode('utf-8'))
            return

        # Collect incoming data
        jsonData = json.loads(self.rfile.read(int(self.headers['Content-Length'])))
        self.log_url()  # Log the URL

        current_time = time.time()

        # Only save data if the defined interval has passed
        if current_time - last_saved_time >= save_interval:
            if start_time is None:
                start_time = current_time
            elapsed_time = round(current_time - start_time, 2)
            data1 = jsonData.get("x_acceleration", [0])
            data2 = jsonData.get("y_acceleration", [0])
            data3 = jsonData.get("z_acceleration", [0])
            accel_data = [data1, data2, data3]  # Collect acceleration data
            
            # Store the data in a list (instead of saving immediately to file)
            collected_data.append([elapsed_time, accel_data[0], accel_data[1], accel_data[2]])

            # Update the data store for GET requests
            dataStore["time"] = elapsed_time
            dataStore["accel"] = accel_data
            last_saved_time = current_time

            # Check if we have collected enough data to make a prediction
            if len(collected_data) >= required_data_length:
                paused = True
                # Save the collected data to CSV
                save_data_to_csv(collected_data)
                collected_data = []  # Reset after saving
                self.send_response(200)
                self.send_header('Content-type', 'application/json')
                self.end_headers()
                response = {"message": "Data saved. Pausing to run predictions."}
                self.wfile.write(json.dumps(response).encode('utf-8'))
                # Trigger the prediction process
                run_predictions()  # Call to prediction function
#                 send_results_to_app_inventor()
            else:
                # Acknowledge that data is still being collected
                self.send_response(200)
                self.send_header('Content-type', 'application/json')
                self.end_headers()
                response = {"message": f"Data collected: {len(collected_data)}. Waiting for more data."}
                self.wfile.write(json.dumps(response).encode('utf-8'))

# Set up and start the HTTP server
def run(server_class=HTTPServer, handler_class=MyHTTPRequestHandler, port=80):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f'Starting HTTP server on port {port}...')
    httpd.serve_forever()

if __name__ == '__main__':
    run()

Starting HTTP server on port 80...
URL visited: ['']


127.0.0.1 - - [24/Oct/2024 04:33:47] "GET / HTTP/1.1" 200 -


URL visited: ['upload']
URL visited: ['upload']


127.0.0.1 - - [24/Oct/2024 04:36:22] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 04:36:22] "GET /upload HTTP/1.1" 404 -


URL visited: ['upload']
URL visited: ['upload']


...

127.0.0.1 - - [24/Oct/2024 04:52:07] "GET /upload HTTP/1.1" 404 -


URL visited: ['upload']
URL visited: ['upload']


127.0.0.1 - - [24/Oct/2024 05:20:22] "GET /upload HTTP/1.1" 404 -
127.0.0.1 - - [24/Oct/2024 05:20:22] "POST /upload HTTP/1.1" 200 -


URL visited: ['upload']
URL visited: ['upload']


127.0.0.1 - - [24/Oct/2024 05:20:25] "GET /upload HTTP/1.1" 404 -
